In [1]:
from scipy.special import erfinv
from collections import OrderedDict
from math import sqrt
import numpy as np

In [2]:
def cdfinv(y):
    """简化的公式，与原NormalCDFInverse等价，且精度更高

    公式推导参见https://www.cnblogs.com/htj10/p/8621771.html
    """
    return sqrt(2) * erfinv(2 * y - 1)

In [3]:
def rankGaussTrafo(dataIn):
    hist = dict()       # hist统计元素的出现频率
    for i in dataIn:
        if i not in hist:
            hist[i] = 1
        else:
            hist[i] += 1

    hist = OrderedDict([t for t in sorted(hist.items(), key=lambda d:d[0])])    # 按照key排序

    trafoMap = dict()
    if len(hist) == 1:      # unary column: trafo all to 0
        trafoMap[list(hist.keys())[0]] = 0.0
    elif len(hist) == 2:    # binary column: trafo to 0 / 1
        trafoMap[list(hist.keys())[0]] = 0.0
        trafoMap[list(hist.keys())[1]] = 1.0
    else:                   # more than 2 unique values
        mean = 0.0
        cnt = 0
        N = len(dataIn)

        for key, value in hist.items():
            rankV = cnt * 1.0 / N       # 累计次数占总次数的比例，取值[0,1]，单调递增（和分布函数F的性质对应）

            rankV = rankV * 0.998 + 1e-3    # 注意到cdfinv(0) = -inf，而cdfinv(1) = inf。这个操作使得rankV限制在[0.001,0.999]，而cdfinv(rankV)限制在[-3.09,3.09]，避免了极端情况的发生

            scale_factor = 0.7      # 使用0.7可以得到原cpp中的结果，对分布有收缩作用。建议使用1.0，可保持std=1.0
            rankV = cdfinv(rankV) * scale_factor     # 将其作为分布函数F的值，逆向求N(0,1)的α分位数

            mean += value * rankV   # value是出现次数，乘以rankV。rankV可以看作是value的权重。注意到hist是按key从小到大排序的，排位越后的数对均值的贡献越大
            trafoMap[key] = rankV   # 记录为trafoMap的值
            cnt += value            # 累计次数
        
        mean /= N

        for key in trafoMap.keys():
            trafoMap[key] -= mean   # 每个rankV减去均值，得到最终trafoMap

    dataOut = dataIn.copy()
    for i in range(len(dataIn)):    # 这里简单地把trafoMap映射到输出
        dataOut[i] = trafoMap[dataIn[i]]
    return dataOut

In [4]:
data = [-19.9378,10.5341,-32.4515,33.0969,24.3530,-1.1830,-1.4106,-4.9431,
        14.2153,26.3700,-7.6760,60.3346,36.2992,-126.8806,14.2488,-5.0821,
        1.6958,-21.2168,-49.1075,-8.3084,-1.5748,3.7900,-2.1561,4.0756,
        -9.0289,-13.9533,-9.8466,79.5876,-13.3332,-111.9568,-24.2531,120.1174]

dataOut = rankGaussTrafo(data)
print(dataOut)

[-0.5517799966896523, 0.4090010366917574, -0.8516215614806684, 0.7725859073647011, 0.6097608726396847, 0.17749425578711678, 0.12237751864416965, -0.042296592392275054, 0.4720838737474867, 0.686977660084494, -0.15498726783224498, 0.9868192248755101, 0.8702983648910773, -2.095563782620048, 0.5386408376293267, -0.09810428241439527, 0.23330194580923708, -0.637388243969859, -1.0013321669392468, -0.21338178582526862, 0.06759883169742091, 0.29018493122708683, 0.012820144750672259, 0.34857944922011036, -0.27380337329691556, -0.4745632092448429, -0.3368862103526449, 1.1365298303340896, -0.4034431742344851, -1.2271028375799966, -0.7351007014962355, 1.362300500974838]


In [5]:
import numpy as np
from collections import Counter, OrderedDict

class RGN:
    '''Rank Gaussian Normalization'''
    def __init__(self, data=None, precision=np.float32):
        #data: 1D array or list
        self._data = data
        self.precision = precision        
        self._output = None
        if self._data is None:
            self._trafo_map = None
        else:
            self.fit_transform(self._data)
    
    @property
    def data(self):
        return self._data
    
    @property
    def output(self):
        return self._output
    
    @property
    def precision(self):
        return self._precision
    
    @precision.setter
    def precision(self, p):
        if not isinstance(p, type):
            raise ValueError('precision must be a data type, e.g.: np.float64')
        self._precision = p
    
    def _RationalApproximation(self, t:float)->float: 
        c = [2.515517, 0.802853, 0.010328]
        d = [1.432788, 0.189269, 0.001308]
        return t - ((c[2]*t + c[1])*t + c[0]) / (((d[2]*t + d[1])*t + d[0])*t + 1.0)

    def _NormalCDFInverse(self, p:float) -> float:

        if (p <= 0.0 or p >= 1.0):
            raise Exception('0<p<1. The value of p was: {}'.format(p))
        if (p < 0.5):
            return -self._RationalApproximation(np.sqrt(-2.0*np.log(p)) )
        return self._RationalApproximation( np.sqrt(-2.0*np.log(1-p)) )

    def _vdErfInvSingle01(self, x:float) -> float:
        if x == 0:
            return 0
        elif x < 0:
            return -self._NormalCDFInverse(-x)*0.7
        else:
            return self._NormalCDFInverse(x)*0.7
    
    def fit_transform(self, dataIn:list) -> dict:
        self.fit(dataIn)
        return self.transform(dataIn)

    def fit(self, dataIn:list):
        self._data = dataIn
        trafoMap = OrderedDict()
        hist = Counter(dataIn)
        if len(hist) == 0:
            pass
        elif len(hist) == 1:
            key = list(hist.keys())[0]
            trafoMap[key] = 0.0
        elif len(hist) == 2:
            keys = sorted(list(hist.keys()))
            trafoMap[keys[0]] = 0.0
            trafoMap[keys[1]] = 1.0
        else:
            N = cnt = 0
            for it in hist:
                N += hist[it]
            assert (N == len(dataIn))
            mean = 0.0
            for it in sorted(list(hist.keys())):
                rankV = cnt / N
                rankV = rankV * 0.998 + 1e-3
                rankV = self._vdErfInvSingle01(rankV)
                assert(rankV >= -3.0 and rankV <= 3.0)
                mean += hist[it] * rankV
                trafoMap[it] = rankV
                cnt += hist[it]
            mean /= N
            for it in trafoMap:
                trafoMap[it] -= mean
        self._trafo_map = trafoMap
        return 

    def _binary_search(self, keys, val):
        start, end = 0, len(keys)-1
        while start+1 < end:
            mid = (start + end) // 2
            if val < keys[mid]:
                end = mid
            else:
                start = mid
        return keys[start], keys[end]

    def transform(self, dataIn:list) -> dict:
        dataOut = []
        trafoMap = self._trafo_map
        keys = list(trafoMap.keys())
        if len(keys) == 0:
            raise Exception('No transfermation map')
        for i in range(len(dataIn)):
            val = dataIn[i]
            trafoVal = 0.0
            if val <= keys[0]:
                trafoVal = trafoMap[keys[0]]
            elif val >= keys[-1]:
                trafoVal = trafoMap[keys[-1]]
            elif val in trafoMap:
                trafoVal = trafoMap[val]
            else:
                lower_key, upper_key = self._binary_search(keys, val)
                x1, y1 = lower_key, trafoMap[lower_key]
                x2, y2 = upper_key, trafoMap[upper_key]

                trafoVal = y1 + (val - x1) * (y2 - y1) / (x2 - x1)
            dataOut.append(trafoVal)
        dataOut = np.asarray(dataOut, dtype=self.precision)
        self._output = dataOut
        return self._output  

if __name__ == '__main__':
    data = [-19.9378,10.5341,-32.4515,33.0969,24.3530,-1.1830,-1.4106,-4.9431,
        14.2153,26.3700,-7.6760,60.3346,36.2992,-126.8806,14.2488,-5.0821,
        1.6958,-21.2168,-49.1075,-8.3084,-1.5748,3.7900,-2.1561,4.0756,
        -9.0289,-13.9533,-9.8466,79.5876,-13.3332,-111.9568,-24.2531,120.1174]
    rgn = RGN(data)
    print(rgn.output)

[-0.5516841   0.4087162  -0.85175323  0.77257425  0.60961264  0.17727302
  0.12224737 -0.04206267  0.4718326   0.6868945  -0.15467337  0.98696357
  0.8703632  -2.0957603   0.53843594 -0.09781568  0.23302603 -0.6373639
 -1.0015435  -0.2130664   0.06760511  0.28988373  0.01296299  0.34827676
 -0.27350584 -0.4744023  -0.33662227  1.1367539  -0.40322557 -1.2273859
 -0.73515284  1.3625963 ]


In [7]:
rank_gauss = RGN()
rank_gauss.fit_transform(data)

array([-0.5516841 ,  0.4087162 , -0.85175323,  0.77257425,  0.60961264,
        0.17727302,  0.12224737, -0.04206267,  0.4718326 ,  0.6868945 ,
       -0.15467337,  0.98696357,  0.8703632 , -2.0957603 ,  0.53843594,
       -0.09781568,  0.23302603, -0.6373639 , -1.0015435 , -0.2130664 ,
        0.06760511,  0.28988373,  0.01296299,  0.34827676, -0.27350584,
       -0.4744023 , -0.33662227,  1.1367539 , -0.40322557, -1.2273859 ,
       -0.73515284,  1.3625963 ], dtype=float32)

In [8]:
from sklearn.preprocessing import QuantileTransformer

In [9]:
QuantileTransformer?